In [ ]:
coords, faces, volume_info = nib.freesurfer.read_geometry(
    'raw_data/FreeSurfer_Surfaces/rh.equi_test0.5.pial',
    read_metadata=True
)

# nibabel can convert SurfaceRAS to voxel using the metadata
from nibabel.freesurfer import MGHImage

# Extract the transformation info
print("Volume info keys:", volume_info.keys())

In [ ]:
# Build the vox2ras transformation from the metadata
# FreeSurfer's TkReg RAS to voxel transformation

volume_shape = np.array(volume_info['volume'])  # [1760, 1280, 1760]
voxelsize = np.array(volume_info['voxelsize'])  # [0.1, 0.1, 0.1]
xras = np.array(volume_info['xras'])  # [-1, 0, 0]
yras = np.array(volume_info['yras'])  # [0, 0, -1]
zras = np.array(volume_info['zras'])  # [0, 1, 0]
cras = np.array(volume_info['cras'])  # [-8.0, 9.95, 7.03]

# Build the vox2ras matrix (converts voxel indices to TkReg RAS)
# Formula: RAS = (voxel - volume_center) * voxelsize * direction_cosines + cras

volume_center = volume_shape / 2.0

# Direction cosine matrix
D = np.column_stack([xras, yras, zras])

# Voxel size diagonal matrix
vox_size_matrix = np.diag(voxelsize)

# Rotation and scaling
rot_scale = D @ vox_size_matrix

# Build 4x4 affine
vox2ras_tkr = np.eye(4)
vox2ras_tkr[:3, :3] = rot_scale
vox2ras_tkr[:3, 3] = cras - rot_scale @ volume_center

print("Vox2RAS TkReg matrix:")
print(vox2ras_tkr)

# Now transform vertex: SurfaceRAS -> voxel
ras2vox_tkr = np.linalg.inv(vox2ras_tkr)
voxel_coords = apply_affine(ras2vox_tkr, ras_position.reshape(1, 3))

print(f"\nVertex {test_vertex_id}:")
print(f"  SurfaceRAS: {ras_position}")
print(f"  Full brain voxel: {voxel_coords[0]}")
print(f"  Freeview: [277, 253, 127]")

In [ ]:
# The native.nii.gz affine (what you've been using)
native_affine = np.array([[-0.1, 0., 0., 80.0],
                          [0., 0., 0.1, -78.05],
                          [0., -0.1, 0., 71.0315],
                          [0., 0., 0., 1.]])

# Surface metadata cras
cras = np.array([-8.0, 9.95, 7.0315])

# FreeSurfer surfaces are in TkReg RAS = Scanner RAS - cras
# So to go from surface coords to scanner RAS:
scanner_ras = ras_position - cras

# Then scanner RAS to voxels:
voxels_native = apply_affine(np.linalg.inv(native_affine), scanner_ras.reshape(1,3))

print(f"Surface RAS: {ras_position}")
print(f"Scanner RAS: {scanner_ras}")
print(f"Native volume voxel: {voxels_native[0]}")

Surface RAS: [  6.28624773 -62.62726593   8.4480381 ]
Scanner RAS: [ 14.28624773 -72.57726593   1.4165381 ]
Native volume voxel: [657.1375227  696.14961899  54.7273407 ]


In [ ]:
# Try adding cras instead
scanner_ras_plus = ras_position + cras

voxels_native_plus = apply_affine(np.linalg.inv(native_affine), scanner_ras_plus.reshape(1,3))

print(f"Surface RAS: {ras_position}")
print(f"Scanner RAS (plus cras): {scanner_ras_plus}")
print(f"Native volume voxel: {voxels_native_plus[0]}")
print(f"Freeview shows: [277, 253, 127] in RH 200um")
print(f"Which in full brain 100um would be approximately: {np.array([277, 253, 127]) * 2 + [263*2, 50*2, 0]}")

In [ ]:
# Freeview shows voxel [277, 253, 127] in RH 200um
# Work backwards to find what Scanner RAS this should be

rh_200um_voxel = np.array([277, 253, 127])

# RH 200um affine
rh_affine = np.array([[-0.2, 0., 0., 53.7],
                      [0., 0., 0.2, -78.05],
                      [0., -0.2, 0., 66.03],
                      [0., 0., 0., 1.]])

# Convert to Scanner RAS
scanner_ras_from_freeview = apply_affine(rh_affine, rh_200um_voxel.reshape(1,3))

print(f"Freeview voxel [277, 253, 127] in RH 200um")
print(f"Should be at Scanner RAS: {scanner_ras_from_freeview[0]}")
print(f"\nSurface file says vertex is at Surface RAS: {ras_position}")
print(f"cras = {cras}")
print(f"\nSurface RAS + cras = {ras_position + cras}")
print(f"Surface RAS - cras = {ras_position - cras}")

Freeview voxel [277, 253, 127] in RH 200um
Should be at Scanner RAS: [ -1.7  -52.65  15.43]

Surface file says vertex is at Surface RAS: [  6.28624773 -62.62726593   8.4480381 ]
cras = [-8.      9.95    7.0315]

Surface RAS + cras = [ -1.71375227 -52.67726593  15.4795381 ]
Surface RAS - cras = [ 14.28624773 -72.57726593   1.4165381 ]


In [ ]:
# Full transformation chain
cras = np.array([-8.0, 9.95, 7.0315])

# Step 1: Surface RAS → Scanner RAS
scanner_ras = ras_position + cras

# Step 2: Scanner RAS → RH 200um voxels (use RH volume's affine directly)
rh_affine = np.array([[-0.2, 0., 0., 53.7],
                      [0., 0., 0.2, -78.05],
                      [0., -0.2, 0., 66.03],
                      [0., 0., 0., 1.]])

rh_voxel = apply_affine(np.linalg.inv(rh_affine), scanner_ras.reshape(1,3))

print(f"Surface RAS: {ras_position}")
print(f"Scanner RAS: {scanner_ras}")
print(f"RH 200um voxel: {rh_voxel[0]}")
print(f"Freeview shows: [277, 253, 127]")
print(f"Match? {np.allclose(np.floor(rh_voxel[0]), [277, 253, 127])}")

Surface RAS: [  6.28624773 -62.62726593   8.4480381 ]
Scanner RAS: [ -1.71375227 -52.67726593  15.4795381 ]
RH 200um voxel: [277.06876135 252.75230949 126.86367035]
Freeview shows: [277, 253, 127]
Match? False
